In [2]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import cv2
import glob
import os

from pyquaternion import Quaternion
from panorama_to_pinhole import *
from Pano_Utils import *

In [3]:
import open3d as o3d
from pyntcloud import PyntCloud
import pye57

# 1. Load panorama

load the panorama image and its corresponding depth data

In [6]:
dataset_path = "../Raw/"
pano_images = glob.glob(dataset_path+"/picture/*.jpg")
pano_scans = glob.glob(dataset_path+"/*.e57")
pano_scans.sort()
print(len(pano_images), len(pano_scans))

266 133


In [43]:
draw_clouds = []

for k in range(len(pano_scans)):
#for k in range(2):    
    points, colors, R_scan, t_scan = read_e57_with_pose(pano_scans[k], False)
    
    point_cloud_o3d = o3d.geometry.PointCloud()
    point_cloud_o3d.points = o3d.utility.Vector3dVector(points[:,0:3])
    point_cloud_o3d.colors = o3d.utility.Vector3dVector(np.asarray(colors) / 255)
    point_cloud_o3d = point_cloud_o3d.voxel_down_sample(voxel_size=0.10)
    
    o3d.io.write_point_cloud("../SubSampled/"+str(k)+".ply", point_cloud_o3d)
    #draw_clouds.append(point_cloud_o3d)
#o3d.visualization.draw_geometries(draw_clouds)

### Make a subsampled full cloud

In [32]:
cloud_files = glob.glob("../SubSampled/*.ply")
cloud_files.sort()

cloud_full = None
for cloud_file in cloud_files:
    cloud_o3d = o3d.io.read_point_cloud(cloud_file)
    if(cloud_full is None):
        cloud_full = cloud_o3d
    else:
        pts_total = np.concatenate((np.asarray(cloud_o3d.points), np.asarray(cloud_full.points)))
        colors_total = np.concatenate((np.asarray(cloud_o3d.colors), np.asarray(cloud_full.colors)))
        
        cloud_full = o3d.geometry.PointCloud()
        cloud_full.points = o3d.utility.Vector3dVector(pts_total)
        cloud_full.colors = o3d.utility.Vector3dVector(colors_total)
        cloud_full = cloud_full.voxel_down_sample(voxel_size=0.10)

In [33]:
o3d.visualization.draw_geometries([cloud_full])
o3d.io.write_point_cloud("../SubSampled_cloud.ply", cloud_full)

True

# 2. TSDF from RGBD

In [7]:
scan_folders_path = "../RGBD/"
scan_folders = glob.glob(scan_folders_path + "garden*")
scan_folders.sort()
print("Found",len(scan_folders), "scan folders.")

Found 133 scan folders.


In [8]:
def line_to_array(line):
    array = []
    for i in line.split(" "):
        if len(i) > 3:
            array.append(i)
    return np.asarray(array).astype(float)

def load_pose_scan(file, scale=1):
    f = open(file)
    pose = np.eye(4)
    for i in range(2):
        line = f.readline()[2:-2]
        pose[i, 0:3] = line_to_array(line)
    line = f.readline()[2:-3]
    pose[2, 0:3] = line_to_array(line)    
    line = f.readline()[1:-1]
    pose[0:3, 3] = line_to_array(line)*scale
    return pose

def load_pose_pinhole(file, idx):
    f = open(file)
    for i in range(idx):
        line = f.readline()
    pose = [float(i) for i in line.split(" ")] 
    return np.asarray(pose[1:]).reshape(4,4)

In [22]:
volume = o3d.integration.ScalableTSDFVolume(
    voxel_length=4.0 / 512.0,
    sdf_trunc=0.04,
    color_type=o3d.integration.TSDFVolumeColorType.RGB8)

num_each = 9
for k in range(len(scan_folders)):
#for k in range(60, 62):
    folder = scan_folders[k]
    pose_scan_file = folder + "/scan_pose.txt"
    pose_scan = load_pose_scan(pose_scan_file, scale = 0.1)
    
    q_y90 = Quaternion(axis = [0,1,0], angle=-np.pi/2)
    pose_tfy = np.eye(4); pose_tfy[0:3,0:3] = q_y90.rotation_matrix

    q_x90 = Quaternion(axis = [1,0,0], angle=-np.pi/2)
    pose_tfx = np.eye(4); pose_tfx[0:3,0:3] = q_x90.rotation_matrix
    
    pose_scan_inv = np.linalg.inv(pose_scan)
    pose_scan = np.dot(pose_scan_inv, np.dot(pose_tfx, pose_tfy))
    
    pose_camera_file = folder + "/camera_local_poses.txt"
    for i in range(num_each):
        pose_camera = np.dot(load_pose_pinhole(pose_camera_file, i+1), np.linalg.inv(pose_scan))
        
        depth_file = folder + "/" + str(i+1) +"_d.npy"
        image_file = folder + "/" + str(i+1) +"_r.jpg"
        color = o3d.io.read_image(image_file)
        depth_py = np.load(depth_file)*100
        depth = o3d.geometry.Image(depth_py.astype('u2'))
        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
                color, depth, depth_trunc=2.0, convert_rgb_to_intensity=False)
        
        volume.integrate(
            rgbd_image,
            o3d.camera.PinholeCameraIntrinsic(640, 480, 500, 500, 320, 240),
            pose_camera)

In [23]:
print("Extract a triangle mesh from the volume and visualize it.")
mesh = volume.extract_triangle_mesh()
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

Extract a triangle mesh from the volume and visualize it.


In [24]:
o3d.io.write_triangle_mesh("garden_mesh.ply", mesh)

True

![image](images/mesh_2.png)
![image](images/mesh_1.png)

# 3. Combine with colmap

#### extract images in the first floor

In [3]:
floor_one_output_folder = "/home/viki/Lucas/garden/floor_one"
floor_one_output_folder_d = "/home/viki/Lucas/garden/floor_one/depth"
floor_one_output_folder_r = "/home/viki/Lucas/garden/floor_one/rgb"
floor_one_pose_file = floor_one_output_folder + "/pose.txt"
os.system("mkdir " + floor_one_output_folder)
os.system("mkdir " + floor_one_output_folder_r)
os.system("mkdir " + floor_one_output_folder_d)

256

### make a mesh for the first floor

In [9]:
volume = o3d.integration.ScalableTSDFVolume(
    voxel_length=4.0 / 512.0,
    sdf_trunc=0.04,
    color_type=o3d.integration.TSDFVolumeColorType.RGB8)

num_each = 9
count_all = 1
for k in range(len(scan_folders)):
#for k in range(60, 62):
    folder = scan_folders[k]
    pose_scan_file = folder + "/scan_pose.txt"
    pose_scan = load_pose_scan(pose_scan_file, scale = 0.1)
    
    q_y90 = Quaternion(axis = [0,1,0], angle=-np.pi/2)
    pose_tfy = np.eye(4); pose_tfy[0:3,0:3] = q_y90.rotation_matrix

    q_x90 = Quaternion(axis = [1,0,0], angle=-np.pi/2)
    pose_tfx = np.eye(4); pose_tfx[0:3,0:3] = q_x90.rotation_matrix
    
    pose_scan_inv = np.linalg.inv(pose_scan)
    pose_scan = np.dot(pose_scan_inv, np.dot(pose_tfx, pose_tfy))
    pose_scan = np.linalg.inv(pose_scan)
    
    if(pose_scan[1,3] > 6.5):
        continue
        
    if(pose_scan[1,3] < 6.2):
        continue
    
    pose_camera_file = folder + "/camera_local_poses.txt"
    for i in range(num_each):
        pose_camera = np.dot(load_pose_pinhole(pose_camera_file, i+1), pose_scan)        
        
        depth_file = folder + "/" + str(i+1) +"_d.npy"
        image_file = folder + "/" + str(i+1) +"_r.jpg"

        color = o3d.io.read_image(image_file)
        depth_py = np.load(depth_file)*100
        depth = o3d.geometry.Image(depth_py.astype('u2'))
        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
                color, depth, depth_trunc=2.0, convert_rgb_to_intensity=False)
        
        volume.integrate(
            rgbd_image,
            o3d.camera.PinholeCameraIntrinsic(640, 480, 500, 500, 320, 240),
            pose_camera)

In [10]:
print("Extract a triangle mesh from the volume and visualize it.")
mesh = volume.extract_triangle_mesh()
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

Extract a triangle mesh from the volume and visualize it.


In [11]:
o3d.io.write_triangle_mesh("garden_mesh_floor1.ply", mesh)

True

### record depth and images for colmap

In [43]:
num_each = 9
file_poses = open(floor_one_pose_file, "w")
count_all = 1
for k in range(len(scan_folders)):
#for k in range(60, 62):
    folder = scan_folders[k]
    pose_scan_file = folder + "/scan_pose.txt"
    pose_scan = load_pose_scan(pose_scan_file, scale = 0.1)
    
    q_y90 = Quaternion(axis = [0,1,0], angle=-np.pi/2)
    pose_tfy = np.eye(4); pose_tfy[0:3,0:3] = q_y90.rotation_matrix

    q_x90 = Quaternion(axis = [1,0,0], angle=-np.pi/2)
    pose_tfx = np.eye(4); pose_tfx[0:3,0:3] = q_x90.rotation_matrix
    
    pose_scan_inv = np.linalg.inv(pose_scan)
    pose_scan = np.dot(pose_scan_inv, np.dot(pose_tfx, pose_tfy))
    pose_scan = np.linalg.inv(pose_scan)
    
    if(pose_scan[1,3] > 6.5):
        continue
        
    if(pose_scan[1,3] < 6.2):
        continue
    
    pose_camera_file = folder + "/camera_local_poses.txt"
    for i in range(num_each):
        pose_camera = np.dot(load_pose_pinhole(pose_camera_file, i+1), pose_scan)
        
        str_pose = str(count_all)
        for k in range(4):
            for j in range(4):
                str_pose  = str_pose + ' ' + str(pose_camera[k,j])[0:8]
        str_pose += '\n'
        file_poses.write(str_pose)
        
        
        depth_file = folder + "/" + str(i+1) +"_d.npy"
        image_file = folder + "/" + str(i+1) +"_r.jpg"
        
        image = cv2.imread(image_file)
        cv2.imwrite(floor_one_output_folder_r + "/" + str(count_all)+".jpg", image)
        depth = np.load(depth_file)
        np.save(floor_one_output_folder_d + "/" + str(count_all)+".npy", depth)
        
        count_all+= 1;
        
file_poses.close()